In [ ]:
import keyring        # for loading api token
import urllib.request # for encoding URL parameters
import pandas as pd   # for handling data frames and json
import os             # for outputting the absolute path of the file containing the data

# Authentication

Please read the Authentication section of the [README](README.ipynb) to set up your acess token for the following code.  <span style="color:red"> WARNING: the below code will load the access token you saved from the README.ipynb</span>

In [ ]:
token = keyring.get_password("system", "canvas_token");
print("Loaded token with %d characters." % (len(token)))

# Configuration

The following configuration does not frequently change.  Input your course ID found in the url of your canvas course.

The course id can be found in the url of the course.  
https://usfca.instructure.com/courses/{course_id}/assignments/{assignment}/submissions

In [ ]:
course_id = int(input("Enter your course here: "))

In [ ]:
assignment_id = int(input("Enter your assignment here: "))

In [ ]:
config = {
    'base_url': 'https://usfca.instructure.com',
    'course_id': course_id,
    'assignment': assignment_id
}

# Request Setup

The following sets up the REST API request to list submissions:

<https://canvas.instructure.com/doc/api/submissions.html#method.submissions_api.index>

In [ ]:
api_format = '{base_url}/api/v1/courses/{course_id}/assignments/{assignment}/submissions'
api_url = api_format.format(**config)

print("URL:", api_url)

In [ ]:
params = {
    'per_page': 100
}

encoded = urllib.parse.urlencode(params)
print("Params:", encoded) # do not output api key 

In [ ]:
rest_call = '{}?access_token={}&{}'.format(api_url, token, encoded)
print("REST call is %d characters." % (len(rest_call)))

# Fetch Data

Fetch the JSON data using pandas.

In [ ]:
data = pd.read_json(rest_call) # gets the data from the given url in json format
print('Loaded {} rows and {} columns.'.format(*data.shape)) # prints the number of rows and columns of the recieved data
# all the columns are based on information about the submissions such as when it is submitted, description, etc.
# the rows are based on each submission listed in the assignment

In [ ]:
# output columns (information about the submission)
print('Columns:', list(data.columns.values))

In [ ]:
data.head(5) # each row is a submission

# Wrangling

Some of the columns could use some wrangling.

In [ ]:
wrangled = data.copy()

In [ ]:
# look at a single entry
print(wrangled.iloc[0]['attachments'][0])

In [ ]:
# extract first element from the list
unwrapped = wrangled.attachments.str[0]
unwrapped

In [ ]:
# split into multiple series (data is any attatchment information per submission for example if a student submitted an image)
attatchments = unwrapped.apply(pd.Series)
attatchments.head(5)

In [ ]:
attatchments.drop(columns=[0], inplace=True) # not useful column

In [ ]:
attatchments.columns = ["attatchment_" + item for item in attatchments.columns.tolist()] # columns may have similar names so rename these

In [ ]:
attatchments.head(5) # make sure columns are renamed

In [ ]:
wrangled = pd.concat([wrangled, attatchments], axis=1) # combine with the original dataframe

In [ ]:
# drop the attachments and submission_comments columns (note that submission comments will have a separate notebook)
wrangled.drop(columns=["attachments"], inplace=True)

In [ ]:
wrangled # check data was combined correctly

# Optional

Do this if you already saved a set of submissions to a file and want to add another set of submissions to it from another assignment

In [ ]:
# submissions = pd.read_csv('submissions.csv')
# wrangled = pd.concat([wrangled, submissions], axis=0, sort=False)

# Output Results

Output the results to CSV.

In [ ]:
path = 'submissions.csv'

In [ ]:
wrangled.to_csv(path, header = True, index = False)

In [ ]:
os.path.abspath('submissions.csv') # the absolute path of the result